# Demo Tree Model

### Dataset: flights.csv
- You'll build a regression model to predict flight delay or not 
- With 'mon', 'dom', 'dow', 'carrier_idx', 'org_idx', 'km', 'depart', 'duration' as a predictor

First thing to do is start a Spark Session

In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark

In [ ]:
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('lr_demo').getOrCreate()

In [ ]:
# Use Spark to read flights.csv file.
data = spark.read.csv("flights.csv",inferSchema=True,header=True)

In [ ]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- mon: integer (nullable = true)
 |-- dom: integer (nullable = true)
 |-- dow: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- org: string (nullable = true)
 |-- mile: integer (nullable = true)
 |-- depart: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- delay: string (nullable = true)



In [ ]:
data.show(3)

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351|   NA|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 3 rows



In [ ]:
data.head()

Row(mon=11, dom=20, dow=6, carrier='US', flight=19, org='JFK', mile=2153, depart=9.48, duration=351, delay='NA')

In [ ]:
# for item in data.head():
#     print(item)

In [ ]:
data.count()

50000

In [ ]:
# Remove the 'flight' column
data = data.drop('flight')

In [ ]:
# Number of records with missing 'delay' values
data.filter('delay IS NULL').count()

0

In [ ]:
# Remove records with missing 'delay' values
data = data.filter('delay IS NOT NULL')

In [ ]:
# Remove records with missing values in any column and get the number of remaining rows
data = data.na.drop()
data.count()

50000

In [ ]:
# Import the required function
from pyspark.sql.functions import round

In [ ]:
# Convert 'mile' to 'km' and drop 'mile' column
data = data.withColumn('km', round(data.mile * 1.60934, 0))

In [ ]:
# Create 'label' column indicating whether flight delayed (1) or not (0)
data = data.withColumn('label', (data.delay >= 15).cast('integer'))
# Check first five records
data.show(3)

+---+---+---+-------+---+----+------+--------+-----+------+-----+
|mon|dom|dow|carrier|org|mile|depart|duration|delay|    km|label|
+---+---+---+-------+---+----+------+--------+-----+------+-----+
| 11| 20|  6|     US|JFK|2153|  9.48|     351|   NA|3465.0| null|
|  0| 22|  2|     UA|ORD| 316| 16.33|      82|   30| 509.0|    1|
|  2| 20|  4|     UA|SFO| 337|  6.17|      82|   -8| 542.0|    0|
|  9| 13|  1|     AA|ORD|1236| 10.33|     195|   -5|1989.0|    0|
|  4|  2|  5|     AA|ORD| 258|  8.92|      65|   NA| 415.0| null|
+---+---+---+-------+---+----+------+--------+-----+------+-----+
only showing top 5 rows



## Categories data

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
# Create an indexer
indexer = StringIndexer(inputCol='carrier', outputCol='carrier_idx')

# Indexer identifies categories in the data
indexer_model = indexer.fit(data)

# Indexer creates a new column with numeric index values
data_indexed = indexer_model.transform(data)

# Repeat the process for the other categorical feature
data_indexed = StringIndexer(inputCol='org', outputCol='org_idx').fit(data_indexed).transform(data_indexed)

In [ ]:
data_indexed.show(3)

+---+---+---+-------+---+----+------+--------+-----+------+-----+-----------+-------+
|mon|dom|dow|carrier|org|mile|depart|duration|delay|    km|label|carrier_idx|org_idx|
+---+---+---+-------+---+----+------+--------+-----+------+-----+-----------+-------+
| 11| 20|  6|     US|JFK|2153|  9.48|     351|   NA|3465.0| null|        6.0|    2.0|
|  0| 22|  2|     UA|ORD| 316| 16.33|      82|   30| 509.0|    1|        0.0|    0.0|
|  2| 20|  4|     UA|SFO| 337|  6.17|      82|   -8| 542.0|    0|        0.0|    1.0|
+---+---+---+-------+---+----+------+--------+-----+------+-----+-----------+-------+
only showing top 3 rows



## Setting Up DataFrame for Machine Learning 

## Assembling columns

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
data_indexed.columns

['mon',
 'dom',
 'dow',
 'carrier',
 'org',
 'mile',
 'depart',
 'duration',
 'delay',
 'km',
 'label',
 'carrier_idx',
 'org_idx']

In [ ]:
# Create an assembler object
assembler = VectorAssembler(inputCols=[
    'mon', 'dom', 'dow', 'carrier_idx', 'org_idx', 'km', 'depart', 'duration'
], outputCol='features')

In [ ]:
data_pre = assembler.transform(data_indexed)

In [ ]:
# Check the resulting column
data_pre.select('features', 'label').show(2, truncate=False)

+-----------------------------------------+-----+
|features                                 |label|
+-----------------------------------------+-----+
|[11.0,20.0,6.0,6.0,2.0,3465.0,9.48,351.0]|null |
|[0.0,22.0,2.0,0.0,0.0,509.0,16.33,82.0]  |1    |
+-----------------------------------------+-----+
only showing top 2 rows



In [ ]:
data_pre.show(3, False)

+---+---+---+-------+---+----+------+--------+-----+------+-----+-----------+-------+--------------------+
|mon|dom|dow|carrier|org|mile|depart|duration|delay|    km|label|carrier_idx|org_idx|            features|
+---+---+---+-------+---+----+------+--------+-----+------+-----+-----------+-------+--------------------+
| 11| 20|  6|     US|JFK|2153|  9.48|     351|   NA|3465.0| null|        6.0|    2.0|[11.0,20.0,6.0,6....|
|  0| 22|  2|     UA|ORD| 316| 16.33|      82|   30| 509.0|    1|        0.0|    0.0|[0.0,22.0,2.0,0.0...|
|  2| 20|  4|     UA|SFO| 337|  6.17|      82|   -8| 542.0|    0|        0.0|    1.0|[2.0,20.0,4.0,0.0...|
|  9| 13|  1|     AA|ORD|1236| 10.33|     195|   -5|1989.0|    0|        1.0|    0.0|[9.0,13.0,1.0,1.0...|
|  4|  2|  5|     AA|ORD| 258|  8.92|      65|   NA| 415.0| null|        1.0|    0.0|[4.0,2.0,5.0,1.0,...|
+---+---+---+-------+---+----+------+--------+-----+------+-----+-----------+-------+--------------------+
only showing top 5 rows



In [ ]:
final_data = data_pre.select("features","label")
final_data.count()

50000

In [ ]:
final_data = final_data.na.drop()
final_data.count()

47022

In [ ]:
final_data.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.0,22.0,2.0,0.0...|    1|
|[2.0,20.0,4.0,0.0...|    0|
|[9.0,13.0,1.0,1.0...|    0|
|[5.0,2.0,1.0,0.0,...|    0|
|[7.0,2.0,6.0,1.0,...|    1|
+--------------------+-----+
only showing top 5 rows



In [ ]:
train_data,test_data = final_data.randomSplit([0.8,0.2])

In [ ]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|              37614|
|   mean| 0.5112192268836072|
| stddev|0.49988075802113874|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [ ]:
test_data.describe().show()

+-------+------------------+
|summary|             label|
+-------+------------------+
|  count|              9408|
|   mean|0.5116921768707483|
| stddev|0.4998898422869004|
|    min|                 0|
|    max|                 1|
+-------+------------------+



# Decision Tree
- ...